In [2]:
## ✅ Objective
# Predict renovation cost using engineered features and job description text.

# ✅ Step 1: Load Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.pipeline import make_pipeline
import sys
import os
sys.path.append(os.path.abspath("../src"))
from cost_estimator import CostEstimator
import joblib

# ✅ Step 2: Load Data
file_path = '../data/processed_data/apartment_cluster_labeled.csv'
df = pd.read_csv(file_path)

# ✅ Step 3: Feature Engineering
# Drop rows with missing cost
cost_df = df[df['Initial Cost'].notna()].copy()

# Clean and convert cost
cost_df['Initial Cost'] = cost_df['Initial Cost'].replace("[\$,]", "", regex=True).astype(float)

# Parse year
cost_df['Permit Year'] = pd.to_datetime(cost_df['Fully Permitted'], errors='coerce').dt.year

# Encode categorical variables
cost_df['Borough Code'] = cost_df['Borough'].astype('category').cat.codes
cost_df['Job Type Code'] = cost_df['Job Type'].astype('category').cat.codes
cost_df['Cluster'] = cost_df['Cluster Label'].astype('category').cat.codes

# ✅ Step 4: Define Features and Target
X_structured = cost_df[['Borough Code', 'Job Type Code', 'Cluster', 'Permit Year']]
y = cost_df['Initial Cost']

# ✅ Step 5: Add TF-IDF from Job Description
X_text = cost_df['Job Description'].fillna("")

# ✅ Step 6: Split Data
X_train_struct, X_test_struct, X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_structured, X_text, y, test_size=0.2, random_state=42
)

# ✅ Step 7: Build Model Pipeline
class CombinedFeatures:
    def __init__(self, tfidf_model):
        self.tfidf = tfidf_model

    def transform(self, text_series, structured_df):
        tfidf_features = self.tfidf.transform(text_series)
        return np.hstack((structured_df.values, tfidf_features.toarray()))

# Fit TF-IDF
tfidf = TfidfVectorizer(stop_words='english', max_features=500)
tfidf.fit(X_train_text)

# Combine features
combiner = CombinedFeatures(tfidf)
X_train_combined = combiner.transform(X_train_text, X_train_struct)
X_test_combined = combiner.transform(X_test_text, X_test_struct)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_combined, y_train)

# ✅ Step 8: Evaluate Model
# ✅ Step 8.1: Define Wrapper Class and Save Final Model

y_pred = model.predict(X_test_combined)
print("\n✅ MAE:", mean_absolute_error(y_test, y_pred))
print("✅ R^2 Score:", r2_score(y_test, y_pred))

# ✅ Step 9: Prepare Mappings and Save CostEstimator Model

borough_map = dict(zip(cost_df['Borough'].unique(), cost_df['Borough Code'].unique()))
jobtype_map = dict(zip(cost_df['Job Type'].unique(), cost_df['Job Type Code'].unique()))
cluster_map = dict(zip(cost_df['Cluster Label'].unique(), cost_df['Cluster'].unique()))

final_model = CostEstimator(model, tfidf, borough_map, jobtype_map, cluster_map)
joblib.dump(final_model, '../models/cost_predictor_rf.pkl')
print("\n✅ CostEstimator model saved successfully.")


✅ MAE: 9259.952453633789
✅ R^2 Score: 0.2345688987973762

✅ CostEstimator model saved successfully.


In [1]:
import sklearn
print(sklearn.__version__)

1.3.2
